In [78]:
import pandas as pd
import json
# from wip import WIP

class WIP():
    def __init__(self, configure):
        
        self.processTime = []
        self.configure = configure
        for i in self.configure.index:
            setattr(self, i, self.configure[i])

        

    def set_process_time(self, processTime):
        self.processTime = processTime
    

    def __str__(self):
        return json.dumps(self.configure.to_dict(), indent=4)
    

    def __repr__(self):
        return json.dumps(self.configure.to_dict(), indent=4)

In [48]:
df = pd.read_excel("./semiconductor_data.xlsx", sheet_name=2, dtype=str)

In [7]:
df

,LOT_ID,OPE_NO,EP,QTY,R_QT,ARRIV_T,STAGE,URGENT_W,RECIPE,CANRUN_TOOL,FOUP
0,LOT001,2000.011,1,8,999.0,300,STAGE,0.5,Y8000,EQP003EQP004EQP005EQP006EQP008EQP009,FOUP0001
1,LOT002,2000.011,2,25,9.2,0,STAGE,7.0,Y5000,EQP001EQP004EQP005EQP007EQP008EQP010,FOUP0002
2,LOT003,2000.011,3,22,10.8,0,STAGE,1.0,Z3000,EQP001EQP003EQP007EQP009EQP010,FOUP0003
3,LOT004,2000.011,4,25,3.3,0,STAGE,10.0,Y2000,EQP001EQP002EQP003EQP004EQP008EQP009EQP010,FOUP0004
4,LOT005,2000.011,5,11,999.0,360,STAGE,0.5,Z3000,EQP001EQP002EQP006EQP007EQP008EQP009,FOUP0005
...,...,...,...,...,...,...,...,...,...,...,...
95,LOT096,2000.011,96,5,8.0,0,STAGE,2.0,Z6000,EQP002EQP003EQP004EQP005EQP008,FOUP0096
96,LOT097,2000.011,97,13,6.0,0,STAGE,4.0,Y8000,EQP007EQP008EQP009EQP010,FOUP0097
97,LOT098,2000.011,98,14,999.0,120,STAGE,0.5,Z3000,EQP001EQP003EQP005EQP007EQP008,FOUP0098
98,LOT099,2000.011,99,12,10.5,0,STAGE,9.0,X4000,EQP001EQP002EQP006EQP009,FOUP0099


In [80]:
jobs = []
for i in range(len(df.values)):
    jobs.append(WIP(df.iloc[i]))

In [67]:
jobs[1]

In [65]:
df.iloc[1].index

Index(['LOT_ID', 'OPE_NO', 'EP', 'QTY', 'R_QT', 'ARRIV_T', 'STAGE', 'URGENT_W',
       'RECIPE', 'CANRUN_TOOL', 'FOUP'],
      dtype='object')

In [55]:
for i in df.iloc[1]:
    print(i)

LOT002
2000.011
2
25
9.2
0
STAGE
7
Y5000
EQP001EQP004EQP005EQP007EQP008EQP010
FOUP0002


In [68]:
RECIPE = pd.read_excel("./semiconductor_data.xlsx")

In [69]:
RECIPE

,EQP_ID,RECIPE,PROCESS_TIME
0,EQP001,X1000,80
1,EQP001,Y2000,140
2,EQP001,Z3000,80
3,EQP001,X4000,120
4,EQP001,Y5000,130
...,...,...,...
95,EQP010,Z6000,120
96,EQP010,X7000,150
97,EQP010,Y8000,110
98,EQP010,Z9000,130


In [73]:
for i in range(len(jobs)):
    jobs[i].set_process_time(RECIPE[RECIPE["RECIPE"] == jobs[i].RECIPE])

In [74]:
jobs[0].processTime

,EQP_ID,RECIPE,PROCESS_TIME
7,EQP001,Y8000,100
17,EQP002,Y8000,120
27,EQP003,Y8000,130
37,EQP004,Y8000,110
47,EQP005,Y8000,130
57,EQP006,Y8000,150
67,EQP007,Y8000,110
77,EQP008,Y8000,150
87,EQP009,Y8000,140
97,EQP010,Y8000,110


In [75]:
jobs[0].RECIPE

'Y8000'

In [81]:
print(jobs[0])

{
    "LOT_ID": "LOT001",
    "OPE_NO": "2000.011",
    "EP": "1",
    "QTY": "8",
    "R_QT": "999",
    "ARRIV_T": "300",
    "STAGE": "STAGE",
    "URGENT_W": "0.5",
    "RECIPE": "Y8000",
    "CANRUN_TOOL": "EQP003EQP004EQP005EQP006EQP008EQP009",
    "FOUP": "FOUP0001"
}
